In [2]:
import numpy as np
import time
import decimal
from decimal import Decimal

### NM

In [3]:
# use to get gradient as np.array
def grad_estimate_np(f, x: np.array, eps: decimal.Decimal):
    grad=np.full(len(x), Decimal(0))
    for i in range(len(x)):
        unit_vector = np.full(len(x), Decimal(0))
        unit_vector[i] = Decimal(1)
        grad[i] = round(float((f(x + (eps * unit_vector)) - f(x)) / np.float64(eps)), 15)
    return np.array(grad, dtype=float)

# use for further calculation of hessian estimate
def grad_estimate(f, x: np.array, eps: decimal.Decimal):
    grad=np.full(len(x), Decimal(0))
    for i in range(len(x)):
        unit_vector = np.full(len(x), Decimal(0))
        unit_vector[i] = Decimal(1)
        grad[i] = (f(x + (eps * unit_vector)) - f(x)) / np.float64(eps)
    return np.array(grad)

In [4]:
def hessian_estimate(f, x: np.array, eps_hess: decimal.Decimal, eps_grad):
    hessian = np.full((len(x), len(x)), Decimal(0))
    for i in range(len(x)):
        unit_vector = np.full(len(x), Decimal(0))
        unit_vector[i] = Decimal(1)
        hessian[:, i] = np.array([round(float(g),15) for g in (np.divide(grad_estimate(f=f, x=(x + (eps_hess * unit_vector)), eps=eps_grad) - grad_estimate(f=f, x=x, eps=eps_grad), np.float64(eps_hess)))])
    return np.array(hessian, dtype=float)

In [5]:
def newton_method(f, grad, hess, x0, max_iter=10000, tol=1e-6, c=0.05, rho=0.5, estimate=False):
    
    x = x0
    num_iter = 0
    e_g = Decimal(0.0000000000001)
    e_h = Decimal(0.00000001)
    
    while num_iter < max_iter:
        if estimate:
            x_dec = np.array([Decimal(val) for val in x])
            gradient = grad(f,x_dec,e_g)
            hessian = hess(f,x_dec,e_h,e_g)
        else:  
            gradient = grad(x)
            hessian = hess(x) 
        
        grad_norm = np.linalg.norm(gradient)
        if grad_norm < tol:
            break
                
        a=np.atleast_2d(hessian)
        #print(a)
        if not is_pos_def(a): #check if hessian is pos. definite, else make it pos. definite
            a = cholesky(a)
            
        b=np.atleast_1d(gradient)
        search_direction = np.linalg.solve(a, -b)
        
        alpha = Backtrack(f=f, x=x, gradient=gradient, pk=search_direction, alpha_zero=1, rho=rho, c=c)

        x = x + alpha * search_direction
        num_iter += 1

    x_star = x
    f_star = f(x_star)
    return x_star, round(float(f_star),15), num_iter, grad_norm

In [6]:
def cholesky(a):
    iters = 100000
    i = 0
    beta = 1
    m = np.amin(np.diag(a))
    if m > 0:
        t = 0
    else:
        t = beta - m
    
    while i < iters:
        #print(a+np.dot(t,np.eye(a.shape[0])))
        try:
            l = np.linalg.cholesky(a+np.dot(t,np.eye(a.shape[0])))
            return l
        except np.linalg.LinAlgError:
            if 2*t > beta:
                t = 2*t
            else:
                t = beta
            i += 1
    return 'fail'

In [7]:
def is_pos_def(A):
    try:
        np.linalg.cholesky(A)
        return True
    except np.linalg.LinAlgError:
        return False

In [8]:
def Backtrack(f, x, gradient, pk, alpha_zero=1, rho=0.5, c=0.5):
    alpha = alpha_zero

    while not f(x + alpha * pk) <= float(f(x)) + c * alpha * np.dot(gradient.T, pk):
        alpha *= rho
        
    return alpha

### Methods for active set

In [9]:
def get_G_c(M, y, x):
    if y.shape[0] == 1:
        G = np.eye(M.shape[1]*2)
        c = np.zeros(shape=(M.shape[1]*2,1))
        
        G[:M.shape[1],:M.shape[1]] = M.T @ M
        for i,xi in enumerate(x[int(x.shape[0]/2):,:]):
            G[M.shape[1]+i,M.shape[1]+i] = xi
        c[:M.shape[1]] = M.T * (-2*y)
        return G, c
    else:
        G = np.eye(M.shape[1]*2)
        c = np.zeros(shape=(M.shape[1]*2,1))
        
        G[:M.shape[1],:M.shape[1]] = M.T @ M
        for i,xi in enumerate(x[int(x.shape[0]/2):,:]):
            G[M.shape[1]+i,M.shape[1]+i] = xi
        c[:M.shape[1]] = M.T @ (-2*y)
        return G, c
        return G,c

In [10]:
def val(x,M,y):
    return (1/2)*np.linalg.norm(M@x-y)**2

In [12]:
def constraints(x):
    # matrix A 
    A = []
    x_len = x.shape[0]
    for i in range(x_len):
        zeros_p = np.zeros(shape=2*x_len)
        zeros_n = np.zeros(shape=2*x_len)
        zeros_p[i] = 1
        zeros_p[i+x_len] = -1
        zeros_n[i] = -1
        zeros_n[i+x_len] = -1
        A.append(zeros_p)
        A.append(zeros_n)
    
    # add sum of z constraint 
    zeros = np.zeros(shape=2*x_len)
    zeros[x_len:] = 1
    A.append(zeros)
    
    A = np.array(A)

    # vector b
    b = np.zeros(shape=(2*x_len+1,1))
    b[-1] = 1
    
    return A,b

In [13]:
def get_working_set_x(A, x, b):
    working_indices = []
    for i,a in enumerate(A):
        if np.isclose(a @ x,b[i]):
            working_indices.append(i)
            
    return working_indices

In [14]:
def get_alpha(x, p, A, b, W_k, tol):
    idx = np.arange(A.shape[0])
    idx_inactive = np.delete(idx,W_k)

    A_inactive = A[idx_inactive]
    b_inactive = b[idx_inactive]
    
    idx_inactive = idx_inactive.reshape(idx_inactive.shape[0], 1)
    
    atp = A_inactive@p
    alpha = (b_inactive  - A_inactive@x) / atp
    alpha = alpha[atp > tol]

    if len(alpha) == 0:
        return 1,-1

    idx_inactive = idx_inactive[atp > tol]
    min_ = np.argmin(alpha)
    
    return alpha[min_],idx_inactive[min_]

In [15]:
def get_p_nullspace(x,G,c,W_k,A,b,tol=1e-6):
    A_act = A[W_k]
    m = A_act.shape[0] 
    
    if m == 0:
        p = np.linalg.solve(G, -c - G@x) 
        return p, []

    n = A_act.shape[1]
    b_act = np.zeros(shape=(m, 1))
    g = -(G@x + c) 

    Q, R = np.linalg.qr(A_act.T, 'complete')
    R = R[:m]
    py = np.linalg.solve(R, b_act)
    Y = Q[:, :m]
    p = np.dot(Y, py)

    if n > m:
        Z = Q[:, m:]
        Zt = Z.T
        
        Gz = np.linalg.multi_dot([Zt, G, Z])

        L = np.linalg.cholesky(Gz)

        gz = np.dot(Zt, np.linalg.multi_dot([G, Y, py]) + g)
        pz = np.linalg.solve(L.T, np.linalg.solve(L, gz))

        p += np.dot(Z, pz)
    
    l = np.linalg.solve(R, np.dot(Y.T, np.dot(G, -p) + g))
    return p, l

In [37]:
def active_set(xk,G,c,A,b,tol=1e-6,maxiter=10):
    W_k = get_working_set_x(A, xk, b)
    x_min = xk
    xs = int(xk.shape[0]/2)

    for it in range(maxiter):
        pk, lambda_ = get_p_nullspace(xk,G,c,W_k,A,b,tol=1e-6)
        pk = np.around(pk,10)

        if not pk.any():
            if all(lamb >= 0 for lamb in lambda_):
                print('Found solution: \n',xk[:xs],' \nafter ',it,' iterations\n')
                return xk
            else:
                j = np.argmin(lambda_)
                W_k = np.delete(W_k, np.where(W_k == W_k[j]))

        else:
            alpha_k, i_b = get_alpha(xk, pk, A, b, W_k, tol)
            if alpha_k < 0:
                alpha_k = 0
            elif alpha_k > 1:
                alpha_k = 1
                i_b = -1
                
            xk = xk + alpha_k*pk
     
            if i_b != -1 and i_b not in W_k:
                W_k = np.concatenate((W_k,np.array([i_b])))

    print('solution after max iterations: \n',xk[:xs],'\n') 
    return xk

### 1D

In [70]:
M1 = np.array([[1,2]])  
print(max(np.linalg.svd(M1)[1]),'\n')
y1 = np.array([4])
x1 = np.array([[0.9],[0.1],     [0.9],[0.1]])
x2 = np.array([[-0.8],[0],        [0.8],[0]])
x3 = np.array([[0],[0],   [0],[0]])

A,b = constraints(x1[:2])

s1 = time.time()
G,c = get_G_c(M1,y1,x1)
x1_sol = active_set(x1,G,c,A,b,tol=1e-6,maxiter=10)
d1 = time.time()
print('Runtime:',(d1-s1)*1000,'ms\n')

s2 = time.time()
G,c = get_G_c(M1,y1,x2)
x2_sol = active_set(x2,G,c,A,b,tol=1e-6,maxiter=10)
d2 = time.time()
print('Runtime:',(d2-s2)*1000,'ms\n')

s3 = time.time()
G,c = get_G_c(M1,y1,x3)
x3_sol = active_set(x3,G,c,A,b,tol=1e-6,maxiter=10)
d3 = time.time()
print('Runtime:',(d3-s3)*1000,'ms\n')

print('Function value x1: ',val(x1_sol[:2], M1, y1))
print('Function value x2: ',val(x2_sol[:2], M1, y1))
print('Function value x3: ',val(x3_sol[:2], M1, y1))

2.23606797749979 

Found solution: 
 [[0.]
 [1.]]  
after  1  iterations

Runtime: 1.0006427764892578 ms

Found solution: 
 [[1.11022302e-16]
 [1.00000000e+00]]  
after  3  iterations

Runtime: 1.0018348693847656 ms

Found solution: 
 [[0.]
 [1.]]  
after  3  iterations

Runtime: 1.0001659393310547 ms

Function value x1:  2.0
Function value x2:  2.0
Function value x3:  2.0


In [59]:
def fct_1(x):
    M = np.array([[1,2]])  
    y = np.array([4])
    x = np.atleast_2d(x)
    x = x.astype(np.float64)
    x = x.reshape(x.shape[1],x.shape[0])
    return (1/2)*np.linalg.norm(M@x-y)**2

In [60]:
x1 = np.array([0.9,0.1])
x2 = np.array([-0.8,0.0])
x3 = np.array([0.0,0.0])

x_star, fval, it, grad_norm = newton_method(fct_1, grad_estimate_np, hessian_estimate, x1, max_iter=100, estimate=True)
print(f'minimum {fval:<4} at x = {x_star} after {it:<4} iterations with remaining gradient norm {grad_norm}')

x_star, fval, it, grad_norm = newton_method(fct_1, grad_estimate_np, hessian_estimate, x2, max_iter=100, estimate=True)
print(f'minimum {fval:<4} at x = {x_star} after {it:<4} iterations with remaining gradient norm {grad_norm}')

x_star, fval, it, grad_norm = newton_method(fct_1, grad_estimate_np, hessian_estimate, x3, max_iter=100, estimate=True)
print(f'minimum {fval:<4} at x = {x_star} after {it:<4} iterations with remaining gradient norm {grad_norm}')

minimum 0.0  at x = [1.61885071 1.19057464] after 5    iterations with remaining gradient norm 2.5964930580465646e-09
minimum 0.0  at x = [0.393712   1.80314401] after 3    iterations with remaining gradient norm 2.0280938873597765e-08
minimum 0.0  at x = [7.50328387e-04 1.99962484e+00] after 3    iterations with remaining gradient norm 5.125110894722182e-10


### 2D

In [72]:
M1 = np.array([[1,2,3,13],[5,6,7,8]]) 
print(max(np.linalg.svd(M1)[1]),'\n')
y1 = np.array([[10],[25]])
x1 = np.array([[0.25],[0.25],[0.25],[0.25],   [0.25],[0.25],[0.25],[0.25]])
x2 = np.array([[0.15],[-0.35],[0.05],[-0.45],   [0.15],[0.35],[0.05],[0.45]])
x3 = np.array([[0.45],[0.25],[0.15],[0.10],    [0.5],[0.25],[0.15],[0.10]])

A,b = constraints(x1[:4])

s1 = time.time()
G,c = get_G_c(M1,y1,x1)
x1_sol = active_set(x1,G,c,A,b,tol=1e-6,maxiter=10)
d1 = time.time()
print('Runtime:',(d1-s1)*1000,'ms\n')

s2 = time.time()
G,c = get_G_c(M1,y1,x2)
x2_sol = active_set(x2,G,c,A,b,tol=1e-6,maxiter=10)
d2 = time.time()
print('Runtime:',(d2-s2)*1000,'ms\n')

s3 = time.time()
G,c = get_G_c(M1,y1,x3)
x3_sol = active_set(x3,G,c,A,b,tol=1e-6,maxiter=10)
d3 = time.time()
print('Runtime:',(d3-s3)*1000,'ms\n')

print('Solutions rounded:\n')
print('x1_sol:\n',np.around(x1_sol[:4],4),'\n')
print('x2_sol:\n',np.around(x1_sol[:4],4),'\n')
print('x3_sol:\n',np.around(x1_sol[:4],4),'\n')

print('Function value x1: ',val(x1_sol[:4], M1, y1))
print('Function value x2: ',val(x2_sol[:4], M1, y1))
print('Function value x3: ',val(x3_sol[:4], M1, y1))

17.904504598680003 

Found solution: 
 [[0.]
 [0.]
 [0.]
 [1.]]  
after  3  iterations

Runtime: 1.0118484497070312 ms

Found solution: 
 [[-2.77555756e-17]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 1.00000000e+00]]  
after  5  iterations

Runtime: 1.4910697937011719 ms

Found solution: 
 [[0.]
 [0.]
 [0.]
 [1.]]  
after  4  iterations

Runtime: 1.001119613647461 ms

Solutions rounded:

x1_sol:
 [[0.]
 [0.]
 [0.]
 [1.]] 

x2_sol:
 [[0.]
 [0.]
 [0.]
 [1.]] 

x3_sol:
 [[0.]
 [0.]
 [0.]
 [1.]] 

Function value x1:  149.00000000012093
Function value x2:  149.00000000118249
Function value x3:  149.00000000036115


In [62]:
def fct_2(x):
    M = np.array([[1,2,3,13],[5,6,7,8]]) 
    y = np.array([[10],[25]])
    x = np.atleast_2d(x)
    x = x.astype(np.float64)
    x = x.reshape(x.shape[1],x.shape[0])
    return (1/2)*np.linalg.norm(M@x-y)**2

In [63]:
x1 = np.array([0.25,0.25,0.25,0.25])
x2 = np.array([0.15,-0.35,0.05,-0.45])
x3 = np.array([0.45,0.25,0.15,0.10])


x_star, fval, it, grad_norm = newton_method(fct_2, grad_estimate_np, hessian_estimate, x1, max_iter=1000, estimate=True)
print(f'minimum {fval:<4} at x = {x_star} after {it:<4} iterations with remaining gradient norm {grad_norm}')

x_star, fval, it, grad_norm = newton_method(fct_2, grad_estimate_np, hessian_estimate, x2, max_iter=1000, estimate=True)
print(f'minimum {fval:<4} at x = {x_star} after {it:<4} iterations with remaining gradient norm {grad_norm}')

x_star, fval, it, grad_norm = newton_method(fct_2, grad_estimate_np, hessian_estimate, x3, max_iter=1000, estimate=True)
print(f'minimum {fval:<4} at x = {x_star} after {it:<4} iterations with remaining gradient norm {grad_norm}')

minimum 1e-15 at x = [ 4.83193212 -0.14898375 -0.31616624  0.4934257 ] after 211  iterations with remaining gradient norm 3.0410229982427376e-07
minimum 0.0  at x = [1.04102033 0.6671314  2.15372678 0.08950281] after 347  iterations with remaining gradient norm 1.1945440510922986e-11
minimum 0.0  at x = [0.53266207 0.37963726 2.85225002 0.01163949] after 123  iterations with remaining gradient norm 2.359335020827424e-07


### 3D

In [83]:
M1 = np.array([[3,3,2,2,1,4],[3,1,2,11,1,0],[4,4,7,8,2,1]]) 
print(max(np.linalg.svd(M1)[1]),'\n')
y1 = np.array([[25],[20],[10]])
x1 = np.array([[0],[0],[0],[0],[0],[0],                         [0],[0],[0],[0],[0],[0]])
x2 = np.array([[0],[0.2],[0.13],[0.16],[0.26],[0.25],           [0],[0.2],[0.13],[0.16],[0.26],[0.25]])
x3 = np.array([[-0.2],[0.3],[0.15],[0.1],[-0.05],[0.2],         [0.2],[0.3],[0.15],[0.1],[0.05],[0.2]])

A,b = constraints(x1[:6])

s1 = time.time()
G,c = get_G_c(M1,y1,x1)
x1_sol = active_set(x1,G,c,A,b,tol=1e-6,maxiter=100)
d1 = time.time()
print('Runtime:',(d1-s1)*1000,'ms\n')

s2 = time.time()
G,c = get_G_c(M1,y1,x2)
x2_sol = active_set(x2,G,c,A,b,tol=1e-6,maxiter=100)
d2 = time.time()
print('Runtime:',(d2-s2)*1000,'ms\n')

s3 = time.time()
G,c = get_G_c(M1,y1,x3)
x3_sol = active_set(x3,G,c,A,b,tol=1e-6,maxiter=10)
d3 = time.time()
print('Runtime:',(d3-s3)*1000,'ms\n')

print('Solutions rounded:\n')
print('x1_sol:\n',np.around(x1_sol[:6],4),'\n')
print('x2_sol:\n',np.around(x2_sol[:6],4),'\n')
print('x3_sol:\n',np.around(x3_sol[:6],4),'\n')

print('Function value x1: ',val(x1_sol[:6], M1, y1))
print('Function value x2: ',val(x2_sol[:6], M1, y1))
print('Function value x3: ',val(x3_sol[:6], M1, y1))

16.847557108740137 

Found solution: 
 [[0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]]  
after  3  iterations

Runtime: 1.0004043579101562 ms

Found solution: 
 [[0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]]  
after  4  iterations

Runtime: 1.5017986297607422 ms

Found solution: 
 [[0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]]  
after  5  iterations

Runtime: 1.5010833740234375 ms

Solutions rounded:

x1_sol:
 [[0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]] 

x2_sol:
 [[0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]] 

x3_sol:
 [[0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]] 

Function value x1:  306.99999999999994
Function value x2:  306.99999999999994
Function value x3:  306.9999999998999


In [84]:
def fct_3(x):
    M = np.array([[3,3,2,2,1,4],[3,1,2,11,1,3],[4,4,7,8,2,1]]) 
    y = np.array([[25],[20],[10]])
    x = np.atleast_2d(x)
    x = x.astype(np.float64)
    x = x.reshape(x.shape[1],x.shape[0])
    return (1/2)*np.linalg.norm(M@x-y)**2

In [85]:
x1 = np.array([0.0,0.0,0.0,0.0,0.0,0.0])
x2 = np.array([0.01,0.2,0.13,0.16,0.2,0.25])
x3 = np.array([-0.2,0.3,0.15,0.1,-0.05,0.2])

x_star, fval, it, grad_norm = newton_method(fct_3, grad_estimate_np, hessian_estimate, x1, max_iter=10000, estimate=True)
print(f'minimum {fval:<4} at x = {x_star} after {it:<4} iterations with remaining gradient norm {grad_norm}')

x_star, fval, it, grad_norm = newton_method(fct_3, grad_estimate_np, hessian_estimate, x2, max_iter=10000, estimate=True)
print(f'minimum {fval:<4} at x = {x_star} after {it:<4} iterations with remaining gradient norm {grad_norm}')

x_star, fval, it, grad_norm = newton_method(fct_3, grad_estimate_np, hessian_estimate, x3, max_iter=10000, estimate=True)
print(f'minimum {fval:<4} at x = {x_star} after {it:<4} iterations with remaining gradient norm {grad_norm}')

minimum 0.0  at x = [ 1.16464483  0.87375102 -0.75813066  0.18442933  0.38276732  4.91236195] after 1192 iterations with remaining gradient norm 3.187242165701285e-07
minimum 1e-15 at x = [ 0.3279871   0.45010149 -0.09849438  0.16016543  0.37724849  5.54128592] after 514  iterations with remaining gradient norm 5.020765952314966e-07
minimum 1e-15 at x = [ 0.919383    1.09175884 -0.72996259  0.23851614  0.09695703  4.96312758] after 1175 iterations with remaining gradient norm 5.249530897212996e-07


### 4D

In [86]:
M1 = np.array([[7,2,1,2,1,3,1,1],[2,1,5,3,1,20,1,1],[1,4,1,2,15,1,9,8],[7,6,5,4,3,2,1,0]]) 
print(max(np.linalg.svd(M1)[1]),'\n')
y1 = np.array([[15],[20],[40],[20]])
x1 = np.array([[0.07],[0.22],[0],[0.08],[0.21],[0.23],[0.18],[0.01],           [0.07],[0.22],[0],[0.08],[0.21],[0.23],[0.18],[0.01]])
x2 = np.array([[-0.1],[0.1],[0.15],[-0.05],[0.15],[0.1],[0.2],[0.1],            [0.1],[0.1],[0.15],[0.05],[0.15],[0.1],[0.2],[0.1]])
x3 = np.array([[0.2],[0],[0.1],[0.05],[0.025],[0.3],[0.1],[-0.2],                [0.2],[0],[0.1],[0.05],[0.025],[0.3],[0.1],[0.2]])

A,b = constraints(x1[:8])

s1 = time.time()
G,c = get_G_c(M1,y1,x1)
x1_sol = active_set(x1,G,c,A,b,tol=1e-6,maxiter=10)
d1 = time.time()
print('Runtime:',(d1-s1)*1000,'ms\n')

s2 = time.time()
G,c = get_G_c(M1,y1,x2)
x2_sol = active_set(x2,G,c,A,b,tol=1e-6,maxiter=10)
d2 = time.time()
print('Runtime:',(d2-s2)*1000,'ms\n')

s3 = time.time()
G,c = get_G_c(M1,y1,x3)
x3_sol = active_set(x3,G,c,A,b,tol=1e-6,maxiter=10)
d3 = time.time()
print('Runtime:',(d3-s3)*1000,'ms\n')

print('Solutions rounded:\n')
print('x1_sol:\n',np.around(x1_sol[:8],4),'\n')
print('x2_sol:\n',np.around(x1_sol[:8],4),'\n')
print('x3_sol:\n',np.around(x1_sol[:8],4),'\n')

print('Function value x1: ',val(x1_sol[:8], M1, y1))
print('Function value x2: ',val(x2_sol[:8], M1, y1))
print('Function value x3: ',val(x3_sol[:8], M1, y1))

23.92636206811054 

Found solution: 
 [[0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]]  
after  6  iterations

Runtime: 1.501321792602539 ms

Found solution: 
 [[ 0.00000000e+00]
 [ 0.00000000e+00]
 [-2.77555756e-17]
 [ 0.00000000e+00]
 [ 1.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]]  
after  8  iterations

Runtime: 2.0017623901367188 ms

Found solution: 
 [[ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 1.00000000e+00]
 [-5.55111512e-17]
 [ 0.00000000e+00]
 [ 0.00000000e+00]]  
after  7  iterations

Runtime: 1.5010833740234375 ms

Solutions rounded:

x1_sol:
 [[0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]] 

x2_sol:
 [[0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]] 

x3_sol:
 [[0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]] 

Function value x1:  735.5
Function value x2:  735.4999999999276
Function value x3:  735.4999999949407


In [87]:
def fct_4(x):
    M = np.array([[7,2,1,2,1,3,1,1],[2,1,5,3,1,20,1,1],[1,4,1,2,15,1,9,8],[7,6,5,4,3,2,1,0]]) 
    y = np.array([[15],[20],[40],[20]])
    x = np.atleast_2d(x)
    x = x.astype(np.float64)
    x = x.reshape(x.shape[1],x.shape[0])
    return (1/2)*np.linalg.norm(M@x-y)**2

In [88]:
x1 = np.array([0.07,0.22,0,0.08,0.21,0.23,0.18,0.01])
x2 = np.array([-0.1,0.1,0.15,-0.85,0.25,0.1,0.2,0.3])
x3 = np.array([0.2,0,0.1,0.05,0.25,0.3,0.1,-0.2])

x_star, fval, it, grad_norm = newton_method(fct_4, grad_estimate_np, hessian_estimate, x1, max_iter=1000, estimate=True)
print(f'minimum {fval:<4} at \nx = {x_star} after {it:<4} iterations with remaining gradient norm {grad_norm}')

x_star, fval, it, grad_norm = newton_method(fct_4, grad_estimate_np, hessian_estimate, x2, max_iter=1000, estimate=True)
print(f'minimum {fval:<4} at \nx = {x_star} after {it:<4} iterations with remaining gradient norm {grad_norm}')

x_star, fval, it, grad_norm = newton_method(fct_4, grad_estimate_np, hessian_estimate, x3, max_iter=1000, estimate=True)
print(f'minimum {fval:<4} at \nx = {x_star} after {it:<4} iterations with remaining gradient norm {grad_norm}')

minimum 0.0  at 
x = [1.11149648 0.65942183 0.12006571 0.40681614 1.18419314 0.60763678
 1.26754764 0.69233194] after 674  iterations with remaining gradient norm 2.6597993963029844e-07
minimum 0.0  at 
x = [ 1.2264886   0.7296443   0.53569987 -0.33001248  1.11061171  0.59113531
  1.16415886  1.03143983] after 650  iterations with remaining gradient norm 2.383860012770464e-07
minimum 0.0  at 
x = [1.16816176 0.45935427 0.23278542 0.41092122 1.38216944 0.5857706
 0.94108057 0.76896949] after 655  iterations with remaining gradient norm 4.045327535598374e-07


### 5D

In [89]:
M1 = np.array([[5,2,4,1,3,1,4,1,5,1],[1,8,1,2,4,4,0,1,9,1],[1,9,1,2,2,1,6,4,1,1],[1,2,3,3,4,4,1,2,3,4],[1,2,3,1,2,3,1,2,3,5]])  
print(max(np.linalg.svd(M1)[1]), '\n')
y1 = np.array([[10],[15],[30],[25],[15]])
x1 = np.array([[0.1],[0.1],[0.1],[0.1],[0.1],[0.1],[0.1],[0.1],[0.1],[0.1],                       [0.1],[0.1],[0.1],[0.1],[0.1],[0.1],[0.1],[0.1],[0.1],[0.1]])
x2 = np.array([[0.1],[0.15],[-0.05],[-0.025],[0.075],[-0.08],[0.01],[-0.01],[0.2],[0.1],          [0.1],[0.15],[0.05],[0.025],[0.075],[0.08],[0.01],[0.01],[0.2],[0.1]])
x3 = np.array([[-0.7],[0.01],[0.01],[0.02],[-0.06],[0.02],[0.01],[-0.07],[0.05],[-0.05],          [0.7],[0.01],[0.01],[0.02],[0.06],[0.02],[0.01],[0.07],[0.05],[0.05]])

A,b = constraints(x1[:10])

s1 = time.time()
G,c = get_G_c(M1,y1,x1)
x1_sol = active_set(x1,G,c,A,b,tol=1e-6,maxiter=10)
d1 = time.time()
print('Runtime:',(d1-s1)*1000,'ms\n')

s2 = time.time()
G,c = get_G_c(M1,y1,x2)
x2_sol = active_set(x2,G,c,A,b,tol=1e-6,maxiter=100)
d2 = time.time()
print('Runtime:',(d2-s2)*1000,'ms\n')

s3 = time.time()
G,c = get_G_c(M1,y1,x3)
x3_sol = active_set(x3,G,c,A,b,tol=1e-6,maxiter=10)
d3 = time.time()
print('Runtime:',(d3-s3)*1000,'ms\n')

print('Solutions rounded:\n')
print('x1_sol:\n',np.around(x1_sol[:10],4),'\n')
print('x2_sol:\n',np.around(x1_sol[:10],4),'\n')
print('x3_sol:\n',np.around(x1_sol[:10],4),'\n')

print('Function value x1: ',val(x1_sol[:10], M1, y1))
print('Function value x2: ',val(x2_sol[:10], M1, y1))
print('Function value x3: ',val(x3_sol[:10], M1, y1))

20.673264754701375 

Found solution: 
 [[0.00000000e+00]
 [1.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [3.46944695e-18]
 [0.00000000e+00]
 [0.00000000e+00]]  
after  9  iterations

Runtime: 3.002643585205078 ms

Found solution: 
 [[0.00000000e+00]
 [1.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [2.77555756e-17]]  
after  10  iterations

Runtime: 3.502368927001953 ms

Found solution: 
 [[0.00000000e+00]
 [1.00000000e+00]
 [0.00000000e+00]
 [5.55111512e-17]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]]  
after  9  iterations

Runtime: 3.5042762756347656 ms

Solutions rounded:

x1_sol:
 [[0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]] 

x2_sol:
 [[0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]] 

x3_sol:
 [[0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]


In [90]:
def fct_5(x):
    M = np.array([[5,2,4,1,3,1,4,1,5,1],[1,8,1,2,4,4,0,1,9,1],[1,9,1,2,2,1,6,4,1,1],[1,2,3,3,4,4,1,2,3,4],[1,2,3,1,2,3,1,2,3,5]])  
    y = np.array([[10],[15],[30],[25],[15]])
    x = np.atleast_2d(x)
    x = x.astype(np.float64)
    x = x.reshape(x.shape[1],x.shape[0])
    return (1/2)*np.linalg.norm(M@x-y)**2

In [96]:
x1 = np.array([0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1])
x2 = np.array([0.1,0.15,-0.05,-0.025,0.075,-0.08,0.01,-0.01,0.2,0.1])
x3 = np.array([-0.7,0.01,0.01,0.02,-0.06,0.02,0.01,-0.07,0.05,-0.05])

x_star, fval, it, grad_norm = newton_method(fct_5, grad_estimate_np, hessian_estimate, x1, max_iter=300000, estimate=True)
print(f'minimum {fval:<4} at \nx = {x_star} after {it:<4} iterations with remaining gradient norm {grad_norm}')

x_star, fval, it, grad_norm = newton_method(fct_5, grad_estimate_np, hessian_estimate, x2, max_iter=300000, estimate=True)
print(f'minimum {fval:<4} at \nx = {x_star} after {it:<4} iterations with remaining gradient norm {grad_norm}')

x_star, fval, it, grad_norm = newton_method(fct_5, grad_estimate_np, hessian_estimate, x3, max_iter=300000, estimate=True)
print(f'minimum {fval:<4} at \nx = {x_star} after {it:<4} iterations with remaining gradient norm {grad_norm}')

minimum 0.480896569836634 at 
x = [-0.43438133  1.00349942  0.21112794  2.21520878  1.85477568  1.2585522
  1.17812461  1.31067649 -1.30417484  0.78349127] after 300000 iterations with remaining gradient norm 2.0292536966304633
minimum 6.851382315830926 at 
x = [-0.38222733  1.22057122  0.2872675   1.44188112  1.17194647  0.94378601
  1.13130646  1.41800369 -0.96163641  1.39765775] after 300000 iterations with remaining gradient norm 7.681516211352052
minimum 6.275309257222061 at 
x = [-0.83627084  1.12702211  0.56863595  1.45522794  1.21182955  0.90914923
  1.3364009   1.35729108 -0.84130899  1.24547354] after 300000 iterations with remaining gradient norm 7.401921433381527


### Additional problem

In [35]:
M1 = np.array([[1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0],
               [0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0]])    
y1 = np.array([[1],[-2],[3],[-4],[5],[-5],[4],[-3],[2],[-1]])

x1 = np.array([[0.05],[0.05],[0.05],[0.05],[0.05],[0.05],[0.05],[0.05],[0.05],[0.05],[0.05],[0.05],[0.05],[0.05],[0.05],[0.05],[0.05],[0.05],[0.05],[0.05],
               [0.05],[0.05],[0.05],[0.05],[0.05],[0.05],[0.05],[0.05],[0.05],[0.05],[0.05],[0.05],[0.05],[0.05],[0.05],[0.05],[0.05],[0.05],[0.05],[0.05]])

x2 = np.array([[0.01],[0.05],[0.05],[0.0],[0.1],[0.05],[0.02],[0.05],[0.01],[0.05],[0.05],[0.0],[0.05],[0.05],[0.03],[0.05],[0.025],[0.05],[0.05],[0.05],
               [0.01],[0.05],[0.05],[0.0],[0.1],[0.05],[0.02],[0.05],[0.01],[0.05],[0.05],[0.0],[0.05],[0.05],[0.03],[0.05],[0.025],[0.05],[0.05],[0.05]])

x3 = np.array([[-0.01],[-0.05],[0.05],[0.0],[-0.1],[-0.05],[0.02],[0.05],[-0.01],[-0.05],[0.05],[0.0],[-0.05],[-0.05],[0.05],[0.05],[-0.05],[-0.05],[0.05],[0.05],
               [0.01],[0.05],[0.05],[0.0],[0.1],[0.05],[0.02],[0.05],[0.01],[0.05],[0.05],[0.0],[0.05],[0.05],[0.05],[0.05],[0.05],[0.05],[0.05],[0.05]])

x4 = np.array([[0.01],[0.01],[0.02],[0.02],[0.03],[0.03],[0.04],[0.04],[0.05],[0.05],[0.05],[0.05],[0.04],[0.04],[0.03],[0.03],[0.02],[0.02],[0.01],[0.01],
               [0.01],[0.01],[0.02],[0.02],[0.03],[0.03],[0.04],[0.04],[0.05],[0.05],[0.05],[0.05],[0.04],[0.04],[0.03],[0.03],[0.02],[0.02],[0.01],[0.01]])

x5 = np.array([[0.0],[0.01],[0.0],[-0.01],[0.0],[0.01],[0.0],[-0.01],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],
               [0.0],[0.01],[0.0],[0.01],[0.0],[0.01],[0.0],[0.01],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0]])

A,b = constraints(x1[:20])


s1 = time.time()
G,c = get_G_c(M1,y1,x1)
x1_sol = active_set_round(x1,G,c,A,b,tol=1e-6,maxiter=1000)
d1 = time.time()
print('Runtime:',(d1-s1)*1000,'ms\n')

s2 = time.time()
G,c = get_G_c(M1,y1,x2)
x2_sol = active_set_round(x2,G,c,A,b,tol=1e-6,maxiter=1000)
d2 = time.time()
print('Runtime:',(d2-s2)*1000,'ms\n')

s3 = time.time()
G,c = get_G_c(M1,y1,x3)
x3_sol = active_set_round(x3,G,c,A,b,tol=1e-6,maxiter=1000)
d3 = time.time()
print('Runtime:',(d3-s3)*1000,'ms\n')

s4 = time.time()
G,c = get_G_c(M1,y1,x4)
x4_sol = active_set_round(x4,G,c,A,b,tol=1e-6,maxiter=1000)
d4 = time.time()
print('Runtime:',(d4-s4)*1000,'ms\n')

s5 = time.time()
G,c = get_G_c(M1,y1,x5)
x5_sol = active_set_round(x5,G,c,A,b,tol=1e-6,maxiter=1000)
d5 = time.time()
print('Runtime:',(d3-s3)*1000,'ms\n')

print('Solutions rounded:\n')
print('x1_sol:\n',np.around(x1_sol[:20],4),'\n')
print('x2_sol:\n',np.around(x2_sol[:20],4),'\n')
print('x3_sol:\n',np.around(x3_sol[:20],4),'\n')
print('x3_sol:\n',np.around(x4_sol[:20],4),'\n')
print('x3_sol:\n',np.around(x5_sol[:20],4),'\n')
print('Function value at x1_sol: ',val(x1_sol[:20], M1, y1))
print('Function value at x2_sol: ',val(x2_sol[:20], M1, y1))
print('Function value at x3_sol: ',val(x3_sol[:20], M1, y1))
print('Function value at x3_sol: ',val(x4_sol[:20], M1, y1))
print('Function value at x3_sol: ',val(x5_sol[:20], M1, y1))

Found solution: 
 [[-1.2500000e-01]
 [-1.2500000e-01]
 [ 6.9388939e-18]
 [ 0.0000000e+00]
 [-1.2500000e-01]
 [-1.2500000e-01]
 [ 0.0000000e+00]
 [ 0.0000000e+00]
 [ 0.0000000e+00]
 [ 0.0000000e+00]
 [ 0.0000000e+00]
 [ 0.0000000e+00]
 [ 1.2500000e-01]
 [ 1.2500000e-01]
 [ 0.0000000e+00]
 [ 0.0000000e+00]
 [ 1.2500000e-01]
 [ 1.2500000e-01]
 [ 0.0000000e+00]
 [ 0.0000000e+00]]  
after  25  iterations

54.50000000000001
Runtime: 11.00921630859375 ms

Found solution: 
 [[-0.20962556]
 [-0.04192511]
 [ 0.        ]
 [ 0.        ]
 [-0.08281928]
 [-0.16563856]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.12474015]
 [ 0.12474015]
 [ 0.        ]
 [ 0.        ]
 [ 0.16700747]
 [ 0.08350373]
 [ 0.        ]
 [ 0.        ]]  
after  25  iterations

54.250005314330075
Runtime: 10.50877571105957 ms

Found solution: 
 [[-2.09841886e-01]
 [-4.19683772e-02]
 [ 3.46944695e-18]
 [ 0.00000000e+00]
 [-8.29047451e-02]
 [-1.65809490e-01]
 [ 0.00000000e+00]
 [

C:\Users\Anwender\AppData\Local\Temp\ipykernel_15592\98238992.py:11: RuntimeWarning: divide by zero encountered in true_divide
  alpha = (b_inactive  - A_inactive@x) / atp
C:\Users\Anwender\AppData\Local\Temp\ipykernel_15592\98238992.py:11: RuntimeWarning: invalid value encountered in true_divide
  alpha = (b_inactive  - A_inactive@x) / atp


In [32]:
def fct_6(x):
    M = np.array([[1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                  [1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                  [0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                  [0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                  [0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0],
                  [0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0],
                  [0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0],
                  [0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0],
                  [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0],
                  [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0]])    
    y = np.array([[1],[-2],[3],[-4],[5],[-5],[4],[-3],[2],[-1]])
    x = np.atleast_2d(x)
    x = x.astype(np.float64)
    x = x.reshape(x.shape[1],x.shape[0])
    return (1/2)*np.linalg.norm(M@x-y)**2

In [34]:
x1 = np.array([0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05])
x2 = np.array([0.01,0.05,0.05,0.0,0.1,0.05,0.02,0.05,0.01,0.05,0.05,0.0,0.05,0.05,0.03,0.05,0.025,0.05,0.05,0.05])
x3 = np.array([-0.01,-0.05,0.05,0.0,-0.1,-0.05,0.02,0.05,-0.01,-0.05,0.05,0.0,-0.05,-0.05,0.05,0.05,-0.05,-0.05,0.05,0.05])
x4 = np.array([0.01,0.01,0.02,0.02,0.03,0.03,0.04,0.04,0.05,0.05,0.05,0.05,0.04,0.04,0.03,0.03,0.02,0.02,0.01,0.01])
x5 = np.array([0.0,0.01,0.0,-0.01,0.0,0.01,0.0,-0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])

x_star, fval, it, grad_norm = newton_method(fct_6, grad_estimate_np, hessian_estimate, x1, max_iter=1000, estimate=True)
print(f'minimum {fval:<4} at \nx = {x_star} after {it:<4} iterations with remaining gradient norm {grad_norm}\n')

x_star, fval, it, grad_norm = newton_method(fct_6, grad_estimate_np, hessian_estimate, x2, max_iter=1000, estimate=True)
print(f'minimum {fval:<4} at \nx = {x_star} after {it:<4} iterations with remaining gradient norm {grad_norm}\n')

x_star, fval, it, grad_norm = newton_method(fct_6, grad_estimate_np, hessian_estimate, x3, max_iter=1000, estimate=True)
print(f'minimum {fval:<4} at \nx = {x_star} after {it:<4} iterations with remaining gradient norm {grad_norm}\n')

x_star, fval, it, grad_norm = newton_method(fct_6, grad_estimate_np, hessian_estimate, x4, max_iter=1000, estimate=True)
print(f'minimum {fval:<4} at \nx = {x_star} after {it:<4} iterations with remaining gradient norm {grad_norm}\n')

x_star, fval, it, grad_norm = newton_method(fct_6, grad_estimate_np, hessian_estimate, x5, max_iter=1000, estimate=True)
print(f'minimum {fval:<4} at \nx = {x_star} after {it:<4} iterations with remaining gradient norm {grad_norm}\n')

minimum 54.01273006251122 at 
x = [ 0.04044484 -0.59887185  0.05        0.05       -0.22668182 -0.36506404
  0.05        0.05        0.11308889 -0.11272629  0.05        0.05
  0.31811829  0.18045308  0.05        0.05        0.12369855  0.40624928
  0.05        0.05      ] after 267  iterations with remaining gradient norm 0.0

minimum 54.00737186970138 at 
x = [-0.25601328 -0.21679737  0.05        0.          0.14601161 -0.5795377
  0.02        0.05        0.21290996 -0.23452799  0.05        0.
  0.12160618  0.33874593  0.03        0.05        0.01843086  0.46835802
  0.05        0.05      ] after 339  iterations with remaining gradient norm 0.0

minimum 54.0052122519683 at 
x = [-0.01373181 -0.51538628  0.05        0.         -0.28828387 -0.23943229
  0.02        0.05       -0.00732519 -0.04814031  0.05        0.
  0.25437054  0.25938614  0.05        0.05       -0.02981359  0.54799422
  0.05        0.05      ] after 152  iterations with remaining gradient norm 0.0

minimum 54.00329596